In [1]:
from fasthtml.common import *
from fasthtml.jupyter import JupyUvi, HTMX
from dataclasses import dataclass
from datetime import datetime
from math import ceil
import pandas as pd

In [2]:
db = database('trfc.db')

In [3]:
app, rt = fast_app()

In [4]:
user_inputs = {
    'min_season': 1921,
    'max_season': 2024,
    'league_tiers': [2, 3, 4, 5],
    'inc_play_offs': 0,
    'generic_comps': ['Anglo-Italian Cup', "Associate Members' Cup", 'FA Cup', 'FA Trophy', "Full Members' Cup", 'League Cup'],
    'pens_as_draw': 0,
    'venues': ['H', 'A', 'N'],
    'min_games': 10,
}

In [5]:
min_season = user_inputs['min_season']
max_season = user_inputs['max_season']
league_tiers = user_inputs['league_tiers']
inc_play_offs = user_inputs['inc_play_offs']
pens_as_draw = user_inputs['pens_as_draw']
venues = user_inputs['venues']
min_games = user_inputs['min_games']

In [6]:
if inc_play_offs == 0:
    po_filter = 'AND COALESCE(c.is_playoff, 0) != 1'
else:
    po_filter = ''

In [7]:
venue_placeholders = ','.join(['?' for _ in venues])

In [8]:
tier_placeholders = ','.join(['?' for _ in user_inputs['league_tiers']]) if user_inputs['league_tiers'] else ''

comp_placeholders = ','.join(['?' for _ in user_inputs['generic_comps']]) if user_inputs['generic_comps'] else ''

In [9]:
tier_comp_filter = ''
if tier_placeholders or comp_placeholders:
    filters = []
    if tier_placeholders:
        filters.append(f'r.league_tier IN ({tier_placeholders})')
    if comp_placeholders:
        filters.append(f'r.generic_comp IN ({comp_placeholders})')
    tier_comp_filter = 'AND (' + ' OR '.join(filters) + ')'

In [40]:
pens_as_draw = 1

query = f'''
    SELECT
        r.opposition,
        COUNT(*) as P,
        COUNT(
            CASE WHEN
                (? = 0 AND (COALESCE(c.is_multi_leg, 0) != 1 AND c.pens_outcome = 'W') OR r.outcome = 'W')
            OR 
                (? = 1 AND r.outcome = 'W')
            THEN 1 END) as W,
        SUM(r.outcome = 'D') as D,
        COUNT(
            CASE WHEN
                (? = 0 AND (COALESCE(c.is_multi_leg, 0) != 1 AND c.pens_outcome = 'L') OR r.outcome = 'L')
            OR 
                (? = 1 AND r.outcome = 'L')
            THEN 1 END) as L,
        SUM(r.goals_for) as GF,
        SUM(r.goals_against) as GA,
        SUM(r.goals_for) - SUM(r.goals_against) as GD,
        ROUND(CAST(COUNT(
            CASE WHEN
                (? = 0 AND COALESCE(c.pens_outcome, r.outcome) = 'W')
            OR 
                (? = 1 AND r.outcome = 'W')
            THEN 1 END) AS FLOAT) / COUNT(*) * 100, 1) as win_pc
    FROM results r
    LEFT JOIN cup_game_details c ON r.game_date = c.game_date
    LEFT JOIN manager_reigns mr ON r.game_date >= mr.mgr_date_from
        AND (r.game_date <= mr.mgr_date_to OR mr.mgr_date_to IS NULL)
    LEFT JOIN managers m ON mr.manager_id = m.manager_id
    LEFT JOIN seasons s ON r.season = s.season
    WHERE s.ssn_start >= ?
        AND s.ssn_start <= ?
        AND r.venue IN ({venue_placeholders})
        {po_filter}
        {tier_comp_filter}
    GROUP BY r.opposition
    HAVING COUNT(*) >= ?
    ORDER BY P DESC
'''

params = [
    pens_as_draw, pens_as_draw, pens_as_draw, pens_as_draw,
    pens_as_draw, pens_as_draw, min_season, max_season,
    *venues
]

if user_inputs['league_tiers']:
    params.extend(user_inputs['league_tiers'])

if user_inputs['generic_comps']:
    params.extend(user_inputs['generic_comps'])

params.append(min_games)

results = db.execute(query, tuple(params))

df = pd.DataFrame(
    results.fetchall(),
    columns=[d[0] for d in results.description]
)

df.query("opposition=='Hartlepool United'")

,opposition,P,W,D,L,GF,GA,GD,win_pc
29,Hartlepool United,56,21,19,16,68.0,58.0,10.0,37.5


In [11]:
df

,opposition,P,W,D,L,GF,GA,GD,win_pc
0,Crewe Alexandra,145,62,35,48,229.0,189.0,40.0,42.8
1,Rochdale,125,59,27,39,238.0,171.0,67.0,47.2
2,Stockport County,122,47,33,42,174.0,167.0,7.0,38.5
3,Wrexham,114,43,27,44,174.0,178.0,-4.0,37.7
4,Halifax Town,111,49,28,34,173.0,147.0,26.0,44.1
...,...,...,...,...,...,...,...,...,...
104,Salford City,11,5,4,2,17.0,10.0,7.0,45.5
105,Leeds United,10,3,1,6,11.0,22.0,-11.0,30.0
106,Manchester City,10,2,4,4,12.0,22.0,-10.0,20.0
107,Orient,10,3,3,4,12.0,14.0,-2.0,30.0


In [33]:
query = f'''
    SELECT 
        r.outcome,
        c.is_multi_leg,
        c.pens_outcome,
        SUM(CASE WHEN r.outcome = 'W' THEN 1 ELSE 0 END) as W,
        SUM(CASE WHEN r.outcome = 'D' THEN 1 ELSE 0 END) as D,
        SUM(CASE WHEN r.outcome = 'L' THEN 1 ELSE 0 END) as L
    FROM results r
    LEFT JOIN cup_game_details c ON r.game_date = c.game_date
    WHERE r.opposition = 'Hartlepool United'
    GROUP BY r.outcome, c.is_multi_leg, c.pens_outcome
'''

results = db.execute(query)

df = pd.DataFrame(
    results.fetchall(),
    columns=[d[0] for d in results.description]
)

df

,outcome,is_multi_leg,pens_outcome,W,D,L
0,D,NaN,None,0,18,0
1,D,0.0,None,0,1,0
2,L,NaN,None,0,0,14
3,L,0.0,None,0,0,1
4,L,1.0,None,0,0,2
5,W,NaN,None,18,0,0
6,W,0.0,None,2,0,0
7,W,1.0,None,1,0,0
8,W,1.0,L,1,0,0
